In [1]:
%matplotlib inline

from utils import *

In [2]:
Traf     = pandas.concat(map( lambda name : pandas.read_hdf(
           name,
    key  = 'table')
           ,H5('traffic')
           )).query(boundaryQ).reset_index(drop=True)

In [3]:
lLobj    =  ['pickup_latitude','pickup_longitude']
Traf     = Traf.round({x:1 for x in lLobj}).groupby(
            lLobj
            ).key.sum().to_frame().assign(
 traffic = lambda DF : (DF.key-DF.key.min())/(DF.key.max()-DF.key.min())
            ).reset_index().rename(
 columns = {'key':'traffic_count'})
Traf.head()

,pickup_latitude,pickup_longitude,traffic_count,traffic
0,39.0,-74.0,1,0.0
1,39.0,-73.7,1,0.0
2,39.0,-73.4,1,0.0
3,39.0,-72.7,1,0.0
4,39.1,-74.4,1,0.0


In [4]:
towns = pandas.read_csv('./data/us.csv').query('state_id=="NY"'
                                      ).drop_duplicates('zips').groupby(['lat','lng']).sum().reset_index()
towns.head()

,lat,lng,county_fips,population,population_proper,density,incorporated,id
0,40.5834,-74.1496,36085,476015.0,476015.0,3155.0,True,1840034032
1,40.5876,-73.7092,36059,0.0,0.0,1485.4,False,1840034033
2,40.5887,-73.6660,36059,33750.0,33750.0,5883.0,True,1840000988
3,40.5894,-73.7296,36059,1922.0,1922.0,1691.0,True,1840005335
4,40.5903,-73.5795,36059,0.0,0.0,796.9,False,1840005251


In [7]:
result=Link2DFS_by_nearestGeo(
	DF1=towns,
	DF2=Traf,
	ColMapper=[['lat','lng'],['pickup_latitude','pickup_longitude']],
	prefix='traff')

In [9]:
result.head()

,lat,lng,county_fips,population,population_proper,density,incorporated,id,traff_pickup_latitude,traff_pickup_longitude,traff_traffic_count,traff_traffic,traff_Dist
0,40.5834,-74.1496,36085,476015.0,476015.0,3155.0,True,1840034032,40.6,-74.1,2708,0.000084,0.052304
1,40.5876,-73.7092,36059,0.0,0.0,1485.4,False,1840034033,40.6,-73.7,1209,0.000037,0.015440
2,40.5887,-73.6660,36059,33750.0,33750.0,5883.0,True,1840000988,40.6,-73.7,1209,0.000037,0.035829
3,40.5894,-73.7296,36059,1922.0,1922.0,1691.0,True,1840005335,40.6,-73.7,1209,0.000037,0.031441
4,40.5903,-73.5795,36059,0.0,0.0,796.9,False,1840005251,40.6,-73.6,380,0.000012,0.022679


In [30]:
result.to_hdf(
            path('','traffic_summary.h5'),
       key='table',
     index=None)

traffic_summary.head()

,lat,lng,county_fips,population,population_proper,density,incorporated,id,geokey,traffic_count,traffic
0,40.6,-74.1,36085,476015.0,476015.0,3155.0,1.0,1840034032,40.6-74.1,2708.0,8.359698e-05
1,40.6,-73.8,36059,0.0,0.0,3391.2,0.0,1840005267,40.6-73.8,816196.0,2.520555e-02
2,40.6,-73.7,432708,59782.0,59782.0,32876.7,7.0,22080087973,40.6-73.7,1209.0,3.730519e-05
3,40.6,-73.6,108177,0.0,0.0,5268.0,0.0,5520015768,40.6-73.6,380.0,1.170419e-05
4,40.6,-73.3,36103,0.0,0.0,267.1,0.0,1840024182,40.6-73.3,26.0,7.720445e-07
